# Линейная алгебра, одномерный `Tensor` (10 баллов)

В этой домашней работе мы попробуем разобраться в том, что же такое векторы, матрицы и тензоры, как они работают и что можно делать с их помощью. Обычно для этих целей используются библиотеки `numpy`, `tensorflow` и `pytorch`, на основании которых написано еще много чего интересного. Можно было бы воспользоваться ими и пробовать вызывать разные методы, пытаясь понять, как они работают. Но это не наш путь, поэтому мы напишем все сами.

Писать мы будет простенький класс `Tensor`, который представляет собой типизированный многомерный массив. В этой работе мы рассмотрим только случаи с тензорами размерности 1, то есть с обычными векторами.

`Tensor` представляет собой простую обертку над стандартным типизированным питоновским массивом. Внутри он хранит значения элементов переданного ему в конструктор списка, для этого используется стандартный питоновский контейнер `array`. Тип хранимых значений -- `float32`, это числа с плавающей точкой, для хранения которых используется четыре байта (32 бита).

## Что уже сделано

Частично функционал класса `Tensor` уже реализован, а именно та его часть, которая отвечает за хранение данных. Сделано это в файле `aurora/tensor.py`. Что уже есть:

1. Конструктор класса, принимающий единственный аргумент `data`;
2. Валидация входящих данных, проверка того, что дан список, состоящий только из чисел.
3. Сохранение переданных в конструктор координат вектора внутри типизированного массива в поле `self.__data`. Двойное нижнее подчеркивание в поле `__data` делает его приватным, после чего его невозможно прочитать снаружи. Если это непонятно, то это неважно :);
4. Атрибут `shape`, который возвращает кортеж, обозначающий размерность тензора. В нашем случае для вектора это кортеж из одного числа -- длины вектора.
5. Магический метод `__repr__`, который красиво форматирует содержимое объекта класса. Если хочется, его вывод можно скопировать и вставить в код, получив копию объекта;
6. Перегружен оператор `==` и реализовано сравнение двух векторов в методе `isclose` (с точностью до флоата). Если есть желание сравнить два вектора, это можно сделать так: `Tensor([1, 2]) == Tensor([1, 2])`.
7. Заготовки под недостающие операции, которые нужно будет реализовать. Они заполнены шаблонным кодом, который кидает исключение `NotImplementedError`.

Кроме того, частично реализован клиентский интерфейс к библиотеке, его можно найти в файле `aurora/ops.py`. Что уже есть:

1. Функция `randn`, которая принимает кортеж с размерностью вектора, после чего создает вектор нужной длины в виде объекта класса `Tensor` (см. выше), заполняя его координаты случайными значениями из нормального распределения $\mathcal{N}(0, 1)$;
2. Заготовки под конструкторы `tensor`, `zeros`, `ones` и функции `add`, `dot`, `norm` и `mul`. См. ниже.

## Что будем делать

Мы умеем хранить векторы, но нам не хватает операций над ними. Наша маленькая библиотека должна уметь складывать и вычитать векторы, домножать их на скаляры и уметь в операцию скалярного произведения. Если тезисно, то реализовать нужно:

* Создание нулевых векторов функцией `zeros`, например размерности 4: `zeros((4,)) == Tensor([0, 0, 0, 0])`;
* Создание векторов из единиц функцией `ones`, например размерности 3: `ones((3,)) == Tensor([1, 1, 1])`;
* Создание заданных векторов функцией `tensor`, например: `tensor((1, 2, 3)) == Tensor([1, 2, 3])`;
* Сложение через метод `add`, например `tensor([1, 2]).add(tensor([3, 4]))`;
* «Магическое» сложение через метод `__add__`, например `tensor([1, 2]) + tensor([3, 4])`;
* Умножение на скаляр через метод `mul`, например `tensor([1, 2]).mul(2)`;
* «Магическое» умножение через метод `__mul__`, например `tensor([1, 2]) * 2`;
* «Магическое» умножение через метод `__rmul__` (оно же коммутативно!), например `2 * tensor([1, 2])`;
* Скалярное произведение векторов через метод `dot`, например `tensor([1, 2]).dot(tensor([3, 4])) == 11`;
* «Магическое» скалярное произведение через метод `__matmut__` и `@`, например `tensor([1, 2]) @ tensor([1, 2]) == 5`;
* Функции `add`, `dot` и `mul`, которые принимают два тензора и повторяют функционал пп. 3, 5, 9.

Вишенкой на торте будут методы вычисления векторных норм (в нашем случае это `l1` и `l2`-нормы). Читаете «норма», а думаете «длина», это гораздо понятнее и привычнее. Зная, как вычислить векторную норму, можно вычислить расстояние между двумя векторами. Для этого нужно реализовать:

* Вычитание через метод `sub`, например `tensor([1, 2]).sub(tensor([3, 4]))`;
* «Магическое» вычитание через метод `__sub__`, например `tensor([1, 2]) - tensor([3, 4])`;
* Вычисление векторной нормы через метод `norm`, например `l1` нормы `tensor([1, 1]).norm(1)`;
* Функцию `norm`, которая принимает тензор и повторяет функционал предыдущего пункта;

Для всего этого написаны тесты, которые помогут вам проверить правильность вашего кода. Что ж, теперь начнем по порядку :)

# Конструкторы для класса `Tensor` (1.5 баллов)

Для начала нужно реализовать конструирующие операции, у нас их всего три: `tensor`, `zeros` и `ones`. После займемся методами с математическими операциями, они чуть сложнее.

## Конструктор `zeros` (0.5 балла)

Реализуйте функцию-конструктор `zeros` в файле `aurora/ops.py`, которая создает заполненный нулями тензор заданной размерности (в нашем случае это нулевой вектор).  

*Подсказка: создайте список из `shape[0]` нулей и передайте его в конструктор `Tensor`.*

После этого, чтобы удостоверится в правильности реализации формул, прогоните тесты для проверки. Это можно сделать как в ячейке ниже, так и в консоли, запустив следующую команду:

`pytest . -k test_zeros -v`

Для запуска в ячейке перед командой введите восклицательный знак (просто запустите ячейку ниже, он там уже есть). Для проверки конечно было бы удобно, если бы все такие ячейки были выполнены.

In [ ]:
!pytest . -k test_zeros -v

Как понять, что все прошло успешно? Есть несколько признаков:

* `PASSED` -- справа от всех тестов написано `PASSED`;
* `[100%]` -- еще правее цифра 100%;   
* В полосе снизу написано `1 passed`, нет слова `failed` и она зеленого цвета, а не красного. 

Если что-то пошло не так, проверьте свой код и посмотрите в тесты еще раз. Повторяйте до сходимости, после чего переходите к следующему пункту.

## Конструктор `ones` (0.5 балла)

Реализуйте функцию-конструктор `ones` в файле `aurora/ops.py`, которая создает заполненный единицами тензор заданной размерности.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_ones -v`

In [ ]:
!pytest . -k test_ones -v

## Конструктор `tensor` (0.5 балла)

Реализуйте функцию-конструктор `tensor` в файле `aurora/ops.py`, которая создает тензор из заданных список координат.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_tensor -v`

In [ ]:
!pytest . -k test_tensor -v

## Конструкторы: примеры использования

Теперь стоит проверить конструкторы, которые вы только что реализовали. Примеры их использования: создание нулевых векторов, векторов из единиц и векторов из списка координат. Все это конечно же создается в виде тензоров размерности 1. Запустите ячейку ниже, чтобы посмотреть, что получилось:

In [ ]:
import aurora as au

a = au.zeros((2,))
b = au.ones((5,))
c = au.tensor([4, 2, 1])

print(a)
print(b)
print(c)
print()
print(au.zeros((4,)))
print(au.ones((3,)))
print(au.tensor([7, 8, 9]))

# Операции над векторами, методы класса `Tensor` (3 балла)

Теперь, когда реализованы несколько необходимых функций-конструкторов для создания тензоров, можно реализовать математические операции над нашими тензорами-векторами. Кажется, что нужно реализовать всего ничего -- сложение двух векторов, умножение вектора на скаляр, скалярное произведение и вычисление нормы вектора. Пойдем по порядку.

## Сложение векторов, метод `add` (1 балл)

Реализуйте сложение двух векторов (двух одномерных тензоров) в методе `add` в файле `aurora/tensor.py`. Сложение -- это простая бинарная операция, которая принимает два вектора и возвращает третий, координаты которого есть сумма соответствующих координат двух исходных векторов. Метод принимает два аргумента: первый -- это `self`, текущий тензор, до его координат можно добраться через `self.__data`. Второй аргумент -- это `other`, эта переменная хранит второе слагаемое-тензор. Аналогично, до его координат можно добраться через `other.__data`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_add -v`

In [ ]:
!pytest . -k test_add -v

## Умножение на скаляр, метод `mul` (1 балл)

Реализуйте умножение вектора на скаляр в методе `mul` в файле `aurora/tensor.py`. Умножение -- это простая бинарная операция, которая принимает на вход вектор и скаляр и возвращает вектор, координаты которого есть произведение соответствующих координат исходного вектора и переданного скаляра. Метод принимает два аргумента: первый -- это `self`, текущий тензор, до его координат можно добраться через `self.__data`. Второй аргумент -- это `scalar`, эта переменная хранит вещественное число-множитель.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_mul -v`

In [ ]:
!pytest . -k test_mul -v

## Скалярное произведение (англ. dot product), метод `dot` (1 балл)

Реализуйте скалярное произведение двух векторов в методе `dot` в файле `aurora/tensor.py`. Скалярное произведение -- это бинарная операция, которая принимает на вход два вектора и возвращает скаляр. Метод принимает два аргумента: первый -- это `self`, текущий тензор, до его координат можно добраться через `self.__data`. Второй аргумент -- это `other`, эта переменная хранит второе слагаемое-тензор. Аналогично, до его координат можно добраться через `other.__data`.

*Подсказка: скалярное произведение -- это просто попарно перемножить координаты, а потом сложить что получилось. Это верно для ортонормированного базиса, а другое нас пока не интересует. Кого интересует -- см. матрица Грама.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_dot -v`

In [ ]:
!pytest . -k test_dot -v

## Примеры использования

Давайте создадим пару векторов и посмотрим, как пользоваться полученными операциями. Еще раз повторим, функции, привязанные к классам, называются методами. Для вызова методов нам нужно создать объект, на котором этим методы будут вызываться. Как видно из примеров ниже, операции сложения и скалярного произведения коммутативны (не зависят от порядка аргументов).

In [ ]:
import aurora as au

a = au.tensor([1, 2, 3])
b = au.tensor([2, 3, 4])

print('Сложение векторов (1, 2, 3) и (2, 3, 4)')
print(a.add(b))
print(b.add(a))
print('\nУмножение вектора (1, 2, 3) на 2 и 3')
print(a.mul(2))
print(a.mul(3))
print('\nСкалярное произведение (1, 2, 3) и (2, 3, 4)')
print(a.dot(b))
print(b.dot(a))

# Перегрузка операторов (2.5 балла)

Основной функционал математических операций над векторами мы уже реализовали, но пользоваться им неудобно, как видно из примера выше. Мы зовем какие то методы, пишем буквы и скобки там, где было бы удобно пользоваться математической нотацией в виде знаков + и \*.

## Магические методы (2 балла)

В питоне для этого есть отдельный механизм, называемый *перегрузкой операторов*. Все операторы некоммутативны, то есть по умолчанию `a + b` не есть `b + a`. Для реализации левоассоциативных операторов нужно реализовать соответствующие методы в классе `Tensor`, а именно `__add__`, `__mul__` и `__matmul__`. Эти методы для перегрузки операторов называются *магическими*. Для реализации правоассоциативных операторов нужно реализовать такие же методы, только с буквой `r` после двойного подчеркивания и до названия, а именно `__radd__`, `__rmul__` и `__rmatmul__`. Нам нужен только `__rmul__`.

*Вопрос: подумайте, почему не нужно реализовать остальные операции?*

Каждый из этих методов, точно так же, как и выше, принимает два аргумента бинарной операции -- векторы или скаляры, в зависимости от контекста. Их значение такое же, как описано выше для методов `add`, `mul` и `dot`.

Кроме того, было бы удобно пользоваться унарным минусом, чтобы создавать противоположный вектор, например `-tensor([1]) == tensor([-1])`. Для этого необходимо реализовать метод `__neg__`, который не принимает аргументов (а зачем они ему?).

*Подсказка: обратите внимание, что оператор `==` для сравнения двух векторов на равенство уже реализован, посмотрите на него. В этой части не нужно писать нового кода, нужно пользоваться тем, что реализовано выше.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*). Должно пройти четыре теста.

`pytest . -k test_magic -v`

In [ ]:
!pytest . -k test_magic -v

## Примеры использования

Давайте создадим пару векторов и посмотрим, как пользоваться полученными операциями. Посмотрите на использование операторов в ячейке ниже. Значок собаки `@` обозначает оператор матричного умножения (скалярного произведения в нашем случае).

*Вопрос: получилось лучше или хуже, добавилось ли читабельности кода?*

In [ ]:
import aurora as au

a = au.tensor([1, 2, 3])
b = au.tensor([2, 3, 4])

print('Сложение векторов (1, 2, 3) и (2, 3, 4)')
print(a + b)
print(b + a)
print('\nУмножение вектора (1, 2, 3) на 2 и 3')
print(a * 2)
print(3 * a)
print('\nСкалярное произведение (1, 2, 3) и (2, 3, 4)')
print(a @ b)
print(b @ a)

## Вычитание векторов, метод `sub` и перегрузка `-` (0.5 балла)

Для удобства вычисления расстояний между векторами нам понадобится операция вычитания векторов. Это ведь явно красивее, чем складывать векторы, где один имеет знак минус?)

Реализуйте вычитание двух векторов (двух одномерных тензоров) в методе `sub` в файле `aurora/tensor.py`. Тут все то же самое, что и в сложении, так что добавить нечего. После этого перегрузите оператор `-` для увеличения читабельности кода.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_sub -v`

In [ ]:
!pytest . -k test_sub -v

# Векторные нормы, метод `norm` (2 балла)

Осталась вишенка на торте, реализовать вычисление норм (длин) векторов. Для этого реализуйте метод `norm`, который будет вычислять $l_1$ и $l_2$ нормы вектора. Этот метод принимает единственный аргумент `ord`, который может принимать значения 1 или 2 в зависимости от типа нормы (1 для манхэттенского расстояния и 2 для евклидовой нормы).

## $l_2$ норма, метрика L2, евклидова норма, `ord=2` (1 балл)

Как много слов в заголовке, да? На самом деле, это все синонимы длины вектора в школьном понимании. Эта норма является геометрическим расстоянием между двумя точками в многомерном пространстве, вычисляемым по теореме Пифагора. Для вычисления этой нормы параметр `ord` должен быть равен единице. Внутри метода просто вычислите норму по формуле:

$||x||_2 = \sqrt{\sum_{i=0}^{n}x_i^2}$

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_norm_l2 -v`

In [ ]:
!pytest . -k test_norm_l2 -v

## $l_1$ норма, метрика L1, манхэттенское расстояние, `ord=1` (1 балл)

Слов в заголовке еще больше, но это не страшно. Для вектора эта норма представляет собой сумму модулей всех его элементов. Для вычисления этой нормы параметр `ord` должен быть равен двойке. Внутри метода просто вычислите норму по формуле:

$||x||_1 = \sum_{i=0}^{n}|x_i|$

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_norm_l1 -v`

In [ ]:
!pytest . -k test_norm_l1 -v

# Клиентский интерфейс (1 балл)

Осталось сделать последний шажок: реализовать все, что мы сделали в виде отдельных функций в пространстве имен верхнего уровня. Ничего хитрого тут нет, надо вставить в заготовки вызовы уже готовых методов, *делегировав* им работу.

Реализуйте все эти методы в файле `aurora/ops.py`, заготовки там готовы. Проверьте себя, запустив тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_ops -v`

In [ ]:
!pytest . -k test_ops -v

## Примеры использования

Использовать эти функции надо так же, как и методы. Это более читабельно (чем использовать методы), все-таки операции бинарные. Но при этом менее читабельно, чем использовать перегруженные операторы.

In [ ]:
import aurora as au

a = au.tensor([1, 2, 3])
b = au.tensor([2, 3, 4])

print(au.add(a, b))
print(au.sub(a, b))
print(au.mul(a, 3))
print(au.dot(a, b))
print(au.norm(a, ord=1))
print(au.norm(a, ord=2))

Финальные тесты можно запустить командой, все должно быть зеленым. Если это не так, то разберитесь что сломалось, и почините нужный тест.

`pytest . -k test_ops -v`

In [ ]:
!pytest . -v

## Бонус: аксиомы линейного пространства

Можно проверить аксиомы линейного пространства, запустив тест `test_axioms` или `test_stress_axioms`. Содержимое этих тестов можно изучить в ячейке ниже (она даже работает).

In [ ]:
import random
import aurora as au

test_shape = (random.randint(5, 10),)
alpha = random.gauss(0, 1)
beta = random.gauss(0, 1)

a, b, c = au.randn(test_shape), au.randn(test_shape), au.randn(test_shape)
zero = au.zeros(test_shape)

assert a + b == b + a
assert (a + b) + c == a + (b + c)
assert a + zero == a
assert a + (-a) == zero

assert alpha * (beta * a) == (alpha * beta) * a
assert (alpha + beta) * a == alpha * a + beta * a
assert alpha * (a + b) == alpha * a + alpha * b
assert 1 * a == a

print('It works!')

# Бонус: расстояние между векторами (1 балл)

Очень полезно при реализации метода k-ближайших соседей. Все, что нужно для вычисления расстояния между двумя векторами, у нас уже реализовано, осталось придумать, как совместить это в едином выражении. Создайте пару векторов и вычислите расстояние между ними, сделав это в ячейке ниже.

*Вопрос: как это сделать?*

*Подсказка: если у вас есть вычисления, тут явно что-то не то.*